# 示例选择器
- 根据长度要求智能选择示例
- 根据输入相似度选择示例(最大)
- 根据输入相似度选择示例（最大余炫相似度）

# 根据长度要求智能选择示例

In [ ]:
# 根据输入的提示词长度综合计算最终长度，智能截取或者添加提示词的示例

In [10]:
from langchain.prompts import PromptTemplate
import os

from dotenv import load_dotenv

load_dotenv()

prompt = PromptTemplate.from_template("你是一个{name}，帮我起一个具有{county}特色的男孩子名字")
prompt.format(name="算命大师",county="中国")


'你是一个算命大师，帮我起一个具有中国特色的男孩子名字'

In [5]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

#假设已经有这么多的提示词示例组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词： {input}\n反义:{output}"
)

#调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    #传入提示词示例组
    examples=examples,
    #提示词模板
    example_prompt=example_prompt,
    #设置格式化后的提示词最大长度
    max_length=25,
    #内置的get_text_length,如果默认分词计算方式不满足，可扩展
    #get_text_length=callable[[std],int] = lambda: x:len(re.split("\n| ",x))

)

#使用小样本提示词模板来实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"],
)

print(dynamic_prompt.format(adjective="big"))

给出每个输入词的反义词

原词： happy
反义:sad

原词： tall
反义:short

原词： sunny
反义:gloomy

原词： windy
反义:calm

原词： 高兴
反义:悲伤

原词：big
反义：


In [6]:
#如果输入长度很长，则最终输出会根据长度要求减少

long_string = "big and huge adn massive and large and gilarge and gilarge and gilarge and gilarge and gilarge and gilarge and "
print(dynamic_prompt.format(adjective=long_string))

给出每个输入词的反义词

原词： happy
反义:sad

原词：big and huge adn massive and large and gilarge and gilarge and gilarge and gilarge and gilarge and gilarge and 
反义：


# 根据输入相似度选择示例(最大边际相关性)
- MMR 是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到平衡
- MMR 会首先找出与输入最相似（即余炫相似度最大）的样本
- 然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本进行惩罚
- MMR 既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡


In [8]:
# 使用MMR来检索相关示例 以使示例尽量符合输入
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS  #FAISS是FACEBOOK大厂出品，品质更优，主要专注于向量相似性搜索，适合处理大规模的高维向量数据 

from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

#构造提示词模板
#假设已经有这么多的提示词示例组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词： {input}\n反义:{output}"
)

#调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_base=openai_api_base,openai_api_key=openai_api_key),
    FAISS,
    #结果条数
    k=2,
)

#使用小样本模板
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"],
)

#当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一对示例组来填充提示词模板
print(mmr_prompt.format(adjective="难过"))   #相似性和多样性兼顾的结果


给出每个输入词的反义词

原词： 高兴
反义:悲伤

原词： tall
反义:short

原词：难过
反义：


# 根据输入相似度选择示例（最大余弦相似度）
- 一种常见的相似度计算方法
- 它通过计算连个向量(在这里，向量可以代表文本，句子或者词语) 之间的余弦值来衡量他们的相似度
- 余弦值越接近1，表示连个向量越相似
- 主要关注的是如何准确衡量连个向量的相似度

In [11]:
# 使用最大余弦相似度检索相关示例，以使示例尽量 符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma # Chromadb则是作为一个嵌入式数据库，更多地服务于构建LLM应用程序，提供嵌入式数据的存储和搜索功能，支持python js 等多语言，开源社区相对比较活跃，对LLM应用更友好更好上手
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

#构造提示词模板
#假设已经有这么多的提示词示例组
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词： {input}\n反义:{output}"
)

#调用MMR
example_selector = SemanticSimilarityExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_base=openai_api_base,openai_api_key=openai_api_key),
    Chroma,
    #结果条数
    k=1,
)

#使用小样本模板
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"],
)

#当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一对示例组来填充提示词模板
print(mmr_prompt.format(adjective="难过"))   #只找最相似的一组



给出每个输入词的反义词

原词： 高兴
反义:悲伤

原词：难过
反义：
